In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import torch.utils.data as data
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from torch.utils.data import ConcatDataset
from torch.cuda.amp import autocast, GradScaler
import os
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
import warnings
import gc
import cv2
# 忽略所有警告
warnings.filterwarnings('ignore')
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
seed_value = 42   # 设定随机数种子

np.random.seed(seed_value)
random.seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)  # 为了禁止hash随机化，使得实验可复现。

torch.manual_seed(seed_value)     # 为CPU设置随机种子
torch.cuda.manual_seed(seed_value)      # 为当前GPU设置随机种子（只用一块GPU）
torch.cuda.manual_seed_all(seed_value)   # 为所有GPU设置随机种子（多块GPU）

torch.backends.cudnn.deterministic = True

class CFG:
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    checpoint = 'result/UnetPlusPlus-DIM-6-[eval_loss]-0.3889-[dice_score]-0.49-10-epoch.pkl'
    # ============== comp exp name =============
    comp_name = 'vesuvius'

    # # comp_dir_path = './'
    # comp_dir_path = '/kaggle/input/'
    # comp_folder_name = 'vesuvius-challenge-ink-detection'
    # # comp_dataset_path = f'{comp_dir_path}datasets/{comp_folder_name}/'
    # comp_dataset_path = f'{comp_dir_path}{comp_folder_name}/'
        # comp_dir_path = './'
    comp_dir_path = ''
    comp_folder_name = 'data'
    # comp_dataset_path = f'{comp_dir_path}datasets/{comp_folder_name}/'
    comp_dataset_path = f'{comp_dir_path}{comp_folder_name}/'
    
    exp_name = 'Unet_stride'

    # ============== pred target =============
    target_size = 1

    # ============== model cfg =============
    model_name = 'Unet'

    in_chans = 24# 65
    # ============== training cfg =============
    size = 224
    tile_size = 224
    stride = tile_size // 2

    train_batch_size = 8 # 32
    valid_batch_size = 8
    use_amp = True

    epochs = 50 # 30

    # lr = 1e-4 / warmup_factor
    lr = 1e-5

    # ============== fixed =============
    pretrained = True

    backbone = 'se_resnext50_32x4d'

    min_lr = 1e-6
    weight_decay = 1e-6
    max_grad_norm = 1000

    num_workers = 4

    seed = 42

    threshhold = 0.5

    all_best_dice = 0
    all_best_loss = np.float('inf')

    shape_list = []
    test_shape_list = []

    val_mask = None
    val_label = None

    # ============== set dataset path =============

    # outputs_path = f'/kaggle/working/outputs/{comp_name}/{exp_name}/'
    outputs_path = 'result/'

    submission_dir = outputs_path + 'submissions/'
    submission_path = submission_dir + f'submission_{exp_name}.csv'

    model_dir = outputs_path
    log_dir = outputs_path + 'logs/'

    # ============== augmentation =============
    train_aug_list = [
        # A.RandomResizedCrop(
        #     size, size, scale=(0.85, 1.0)),
        A.Resize(size, size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.75),
        A.ShiftScaleRotate(p=0.75),
        A.OneOf([
                A.GaussNoise(var_limit=[10, 50]),
                A.GaussianBlur(),
                A.MotionBlur(),
                ], p=0.4),
        A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
        A.CoarseDropout(max_holes=1, max_width=int(size * 0.3), max_height=int(size * 0.3), 
                        mask_fill_value=0, p=0.5),
        # A.Cutout(max_h_size=int(size * 0.6),
        #          max_w_size=int(size * 0.6), num_holes=1, p=1.0),
        A.Normalize(
            mean= [0] * in_chans,
            std= [1] * in_chans
        ),
        ToTensorV2(transpose_mask=True),
    ]

    valid_aug_list = [
        A.Resize(size, size),
        A.Normalize(
            mean= [0] * in_chans,
            std= [1] * in_chans
        ),
        ToTensorV2(transpose_mask=True),
    ]
seed = CFG.seed
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
def read_image_mask(fragment_id):

    images = []

    # idxs = range(65)
    mid = 65 // 2
    start = mid - CFG.in_chans // 2
    end = mid + CFG.in_chans // 2
    idxs = range(start, end)

    for i in tqdm(idxs):
        
        image = cv2.imread(CFG.comp_dataset_path + f"train/{fragment_id}/surface_volume/{i:02}.tif", 0)

        pad0 = (CFG.tile_size - image.shape[0] % CFG.tile_size + 1)
        pad1 = (CFG.tile_size - image.shape[1] % CFG.tile_size + 1)

        image = np.pad(image, [(0, pad0), (0, pad1)], constant_values=0)

        images.append(image)
    images = np.stack(images, axis=2)

    label = cv2.imread(CFG.comp_dataset_path + f"train/{fragment_id}/inklabels.png", 0)
    label = np.pad(label, [(0, pad0), (0, pad1)], constant_values=0)

    label = label.astype('float32')
    label /= 255.0

    mask = cv2.imread(CFG.comp_dataset_path + f"train/{fragment_id}/mask.png", 0)
    mask = np.pad(mask, [(0, pad0), (0, pad1)], constant_values=0)

    mask = (mask / 255.).astype('float32')

    CFG.shape_list.append(mask.shape)
    if fragment_id == 1:
        CFG.val_mask = mask
        CFG.val_label = label
    
    return images, label, mask

def get_train_valid_dataset(val_persent=0.05):
    train_id = [2, 3]
    train_images = []
    train_labels = []

    valid_images = []
    valid_labels = []
    valid_positons = []

    for fragment_id in range(1, 4):

        image, label, mask = read_image_mask(fragment_id)

        x1_list = list(range(0, image.shape[1]-CFG.tile_size+1, CFG.stride))
        y1_list = list(range(0, image.shape[0]-CFG.tile_size+1, CFG.stride))

        for y1 in y1_list:
            for x1 in x1_list:
                y2 = y1 + CFG.tile_size
                x2 = x1 + CFG.tile_size
                if sum(list(mask[y1:y2, x1:x2].flatten())) != 0:
                    if fragment_id in train_id:
                        train_images.append(image[y1:y2, x1:x2])
                        train_labels.append(label[y1:y2, x1:x2, None])
                    else:
                        valid_images.append(image[y1:y2, x1:x2])
                        valid_labels.append(label[y1:y2, x1:x2, None])
                        valid_positons.append([x1, y1, x2, y2, fragment_id - 1])
    return train_images, train_labels, valid_images, valid_labels, valid_positons

def get_transforms(data, cfg):
    if data == 'train':
        aug = A.Compose(cfg.train_aug_list)
    elif data == 'valid':
        aug = A.Compose(cfg.valid_aug_list)
    return aug

class SubvolumeDataset(data.Dataset):
    def __init__(self, images, labels, positions, transform, is_train):
        self.transform = transform
        self.images = images
        self.labels = labels
        self.is_train = is_train
        self.positions = positions
    def __len__(self):
        return len(self.images)
    def __getitem__(self, index):
        if self.is_train:
            image = self.images[index]
            label = self.labels[index]
            if self.positions:
                position = np.array(self.positions[index])
            else:
                position = np.zeros(1)
            if self.transform:
                data = self.transform(image=image, mask=label)
                image = data['image']
                label = data['mask']
            return image, label, position
        else:
            image = self.images[index]
            position = np.array(self.positions[index])
            if self.transform:
                data = self.transform(image=image, mask=label)
                image = data['image']
            return image, position
        
# IOU and Dice Score
def dice_coef(targets, preds, thr=0.5, beta=0.5, smooth=1e-5):

    #comment out if your model contains a sigmoid or equivalent activation layer
    # flatten label and prediction tensors
    preds = (preds > thr).view(-1).float()
    targets = targets.view(-1).float()

    y_true_count = targets.sum()
    ctp = preds[targets==1].sum()
    cfp = preds[targets==0].sum()
    beta_squared = beta * beta

    c_precision = ctp / (ctp + cfp + smooth)
    c_recall = ctp / (y_true_count + smooth)
    dice = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall + smooth)

    return dice

In [3]:
train_images, train_labels, valid_images, valid_labels, valid_positons = get_train_valid_dataset()

100%|██████████| 24/24 [00:08<00:00,  2.67it/s]


In [4]:
train_dataset = SubvolumeDataset(train_images, train_labels, None,get_transforms(data='train', cfg=CFG), True)
valid_dataset = SubvolumeDataset(valid_images, valid_labels, valid_positons, get_transforms(data='valid', cfg=CFG), True)
train_loader = data.DataLoader(train_dataset,
                          batch_size=CFG.train_batch_size,
                          shuffle=True,
                          num_workers=CFG.num_workers, pin_memory=True, drop_last=True,
                          )
valid_loader = data.DataLoader(valid_dataset,
                          batch_size=CFG.valid_batch_size,
                          shuffle=False,
                          num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

In [5]:
# model = Ringed_Res_Unet(n_channels=CFG.in_chans, n_classes=CFG.target_size).to(CFG.device)
model = smp.UnetPlusPlus(in_channels=CFG.in_chans, 
                         classes=1, 
                         encoder_name=CFG.backbone, 
                         encoder_weights='imagenet', 
                         activation=None, 
                         decoder_attention_type='scse').to(CFG.device)
# x = np.zeros((16, CFG.in_chans, 224, 224))
# x = torch.from_numpy(x).to(CFG.device).float()
# out = model(x)
from torchsummary import summary
summary(model, input_size=(CFG.in_chans, 224, 224))
model_name = 'UnetPlusPlus'
if CFG.pretrained:
    try:
        checkpoint = torch.load(CFG.checpoint, map_location=CFG.device)
        models_dict = model.state_dict()
        for model_part in models_dict:
            if model_part in checkpoint:
                models_dict[model_part] = checkpoint[model_part]
        model.load_state_dict(models_dict)
        print('Checkpoint loaded')
    except:
        print('Checkpoint not loaded')
        pass

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]          75,264
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5          [-1, 128, 56, 56]           8,192
       BatchNorm2d-6          [-1, 128, 56, 56]             256
              ReLU-7          [-1, 128, 56, 56]               0
            Conv2d-8          [-1, 128, 56, 56]           4,608
       BatchNorm2d-9          [-1, 128, 56, 56]             256
             ReLU-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          32,768
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [6]:
from warmup_scheduler import GradualWarmupScheduler


class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    """
    https://www.kaggle.com/code/underwearfitting/single-fold-training-of-resnet200d-lb0-965
    """
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(
            optimizer, multiplier, total_epoch, after_scheduler)

    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [
                        base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

def get_scheduler(cfg, optimizer):
    scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, cfg.epochs, eta_min=1e-7)
    scheduler = GradualWarmupSchedulerV2(
        optimizer, multiplier=10, total_epoch=1, after_scheduler=scheduler_cosine)

    return scheduler

def scheduler_step(scheduler, avg_val_loss, epoch):
    scheduler.step(epoch)

In [7]:
def train_step(train_loader, model, criterion, optimizer, writer, device, epoch):
    model.train()
    epoch_loss = 0
    scaler = GradScaler(enabled=CFG.use_amp)
    bar = tqdm(enumerate(train_loader), total=len(train_loader)) 
    for step, (image, label, _) in bar:
        optimizer.zero_grad()
        outputs = model(image.to(device))
        loss = criterion(outputs, label.to(device))
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        scaler.step(optimizer)
        scaler.update()
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        bar.set_postfix(loss=f'{loss.item():0.4f}', epoch=epoch ,gpu_mem=f'{mem:0.2f} GB', lr=f'{optimizer.state_dict()["param_groups"][0]["lr"]:0.2e}')
        epoch_loss += loss.item()
    writer.add_scalar('Train/Loss', epoch_loss / len(train_loader), epoch)
    return epoch_loss / len(train_loader)

def valid_step(valid_loader, model, criterion, device, writer, epoch):
    pred_label = np.zeros(CFG.shape_list[0])
    true_label = CFG.val_label
    mask_count = np.zeros(CFG.shape_list[0])
    model.eval()
    epoch_loss = 0
    for step, (images, labels, positions) in tqdm(enumerate(valid_loader), total=len(valid_loader)):
        images = images.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            y_preds = model(images)
            loss = criterion(y_preds, labels)
        # make whole mask
        y_preds = torch.sigmoid(y_preds)
        pred_img =y_preds.squeeze().cpu().numpy()
        positions = positions.squeeze()
        for i in range(len(positions)):
            x1, y1, x2, y2, _ = positions[i].numpy().tolist()
            pred_label[y1:y2, x1:x2] += pred_img[i]
            mask_count[y1:y2, x1:x2] += np.ones((CFG.tile_size, CFG.tile_size))
        epoch_loss += loss.item()
    avg_loss = epoch_loss / len(valid_loader)
    print(f'mask_count_min: {mask_count.min()}')
    print(f'mask_count_max: {mask_count.max()}')
    # 计算准确率
    pred_label /= mask_count
    pred_label *= CFG.val_mask
    best_th = 0
    best_dice = 0
    for th in np.arange(1, 6, 0.5) / 10:
        dice_score = dice_coef(torch.from_numpy(true_label).to(CFG.device), torch.from_numpy(pred_label).to(CFG.device), thr=th).item()
        # dice_scores.append(dice_score)
        if dice_score > best_dice:
            best_dice = dice_score
            best_th = th
    if CFG.all_best_dice < best_dice:
        print('best_th={:2f}' .format(best_th),"score up: {:2f}->{:2f}".format(CFG.all_best_dice, best_dice))       
        cv2.imwrite('result/logs/img/'+str(epoch) + '_res.png', ((pred_label > best_th).astype('int')* 255) )
        CFG.all_best_dice = best_dice
        torch.save(model.state_dict(), 'result/' +  '{}-DIM-{}-[eval_loss]-{:.4f}-[dice_score]-{:.2f}-'.format(model_name, CFG.in_chans , avg_loss, best_dice) + str(epoch) + '-epoch.pkl')
        # 使用make_grid将图片转换成网格形式
        # pred_mask = make_grid(torch.from_numpy(pred_label ).to(CFG.device),normalize=True)
        # true_mask = make_grid(torch.from_numpy(true_label).to(CFG.device), normalize=True)
        # 使用add_image方法将图片添加到TensorBoard中
        # writer.add_image('Valid/True_mask', true_mask, global_step=epoch, dataformats="CHW")
        # writer.add_image('Valid/Pred_mask', pred_mask, global_step=epoch, dataformats="CHW")
    if CFG.all_best_loss > avg_loss:
        print('best_loss={:2f}'.format(avg_loss), "loss down: {:2f}->{:2f}".format(CFG.all_best_loss, avg_loss))
        cv2.imwrite('result/logs/img/' + str(epoch) + '_res.png', ((pred_label > best_th).astype('int')* 255) )
        CFG.all_best_loss = avg_loss
        torch.save(model.state_dict(), 'result/' +  '{}-DIM-{}-[eval_loss]-{:.4f}-[dice_score]-{:.2f}-'.format(model_name, CFG.in_chans ,avg_loss, best_dice) + str(epoch) + '-epoch.pkl')
    writer.add_scalar('Val/Dice', best_dice, epoch)
    writer.add_scalar('Valid/Loss', avg_loss , epoch)
    return avg_loss

In [8]:
criterion = smp.losses.SoftBCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(),
                        lr=CFG.lr,
                        betas=(0.9, 0.999),
                        weight_decay=CFG.weight_decay
                        )
scheduler = get_scheduler(CFG, optimizer)
writer = SummaryWriter('result/logs')
for i in range(CFG.epochs):
    print('train:')
    train_step(train_loader, model, criterion, optimizer, writer, CFG.device, i + 1)
    print('val:')
    val_loss = valid_step(valid_loader, model, criterion, CFG.device, writer, i + 1)
    scheduler_step(scheduler, val_loss, i + 1)

train:


100%|██████████| 1320/1320 [09:33<00:00,  2.30it/s, epoch=1, gpu_mem=5.60 GB, loss=0.3506, lr=1.00e-05]

val:



100%|██████████| 328/328 [00:38<00:00,  8.49it/s]


mask_count_min: 0.0
mask_count_max: 4.0
best_th=0.400000 score up: 0.000000->0.280372
best_loss=0.478190 loss down: inf->0.478190
train:


...

mask_count_min: 0.0
mask_count_max: 4.0
train:


100%|██████████| 1320/1320 [09:46<00:00,  2.25it/s, epoch=50, gpu_mem=7.05 GB, loss=0.0488, lr=4.94e-07]

val:



100%|██████████| 328/328 [00:38<00:00,  8.46it/s]


mask_count_min: 0.0
mask_count_max: 4.0
